# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [57]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Set Up Environment

In [1]:
network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'custom_contracts_20201110125748.pkl'
#contracts_20201109120425

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [3]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [4]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"


with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [4]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [5]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [40]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

78734389964340600000000000

### Importing wallet from keystore file

In [41]:
# Set to the address of the keystore file and give the password when prompted
password = getpass()
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
deployer_wallet.get_address()
icon_service.get_balance(deployer_wallet.get_address())

708876575999000000000000000

### Creating new wallet

In [ ]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

In [6]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
#Run this when you need to zip the contract codes (before deploying them)
contracts=AutoVivification()

deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token']
# for directory in {"score"}:
directory=os.getcwd()
with os.scandir(directory) as it:
    for file in it:
        archive_name = directory + "/" + file.name
        if file.is_dir() and file.name in deploy:
            make_archive(archive_name, "zip", directory, file.name)
            contracts[file.name]['zip'] = archive_name + '.zip'


contracts

{'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider.zip'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool.zip'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider.zip'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore.zip'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken.zip'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle.zip'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/sample_token.zip'},
 'addressProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/addressProvider.zip'}}

In [43]:
# params = {}
# deploy_transaction = DeployTransactionBuilder()\
#     .from_(deployer_wallet.get_address())\
#     .to(GOVERNANCE_ADDRESS)\
#     .nid(3)\
#     .nonce(100)\
#     .content_type("application/zip")\
#     .content(gen_deploy_data_content('addressProvider'))\
#     .params(params)\
#     .build()

# estimate_step = icon_service.estimate_step(deploy_transaction)
# step_limit = estimate_step + 100000
# signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

# tx_hash = icon_service.send_transaction(signed_transaction)

# get_tx_result(tx_hash)
# res = get_tx_result(tx_hash)
# contracts[score]["SCORE"] = res.get('scoreAddress', '')
# now = datetime.utcnow()
# filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
# filename=network+"_"+filename
# print(f'saving contracts list as {filename}')
# with open(filename, 'wb') as f:
#     pkl.dump(contracts, f)


In [42]:


pprint(contracts)


{'addressProvider': {'SCORE': 'cx42dbafcf33b26b0cc7f0e1f7ca377bfeebc6c42e',
                     'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/addressProvider.zip'},
 'feeProvider': {'SCORE': 'cxd66f32f94600843c9441f504ba6ee075d4387bc8',
                 'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider.zip'},
 'lendingPool': {'SCORE': 'cx83e2b9cfa3bc5e0836e8ea48dc4b67f47723a1b1',
                 'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool.zip'},
 'lendingPoolCore': {'SCORE': 'cx5950ae341b07b9d557ed545e45e4f2e3a506ad9a',
                     'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore.zip'},
 'lendingPoolDataProvider': {'SCORE': 'cxf6102cd8af9c3facd6607c286bd14630de92d976',
                             'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider.zip'},
 'oToken': {'SCORE': 'cx5e71af8b721dfe96af95f03cb514bfa04480180a',
            'zip': '/home/shubham/Work

# Deploying all contracts


In [8]:
#this cell needs to run only when you need to deploy new set of contracts
for score, loc in contracts.items():
    params = {}
    print(score,loc)
    if score == "sample_token":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif score == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
filename=network+'_'+filename
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

lendingPoolDataProvider {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider.zip'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
lendingPool {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool.zip'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
feeProvider {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider.zip'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
lendingPoolCore {'zip': '/home/shubham/Work

{'lendingPoolDataProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolDataProvider.zip',
  'SCORE': 'cxf6102cd8af9c3facd6607c286bd14630de92d976'},
 'lendingPool': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPool.zip',
  'SCORE': 'cx83e2b9cfa3bc5e0836e8ea48dc4b67f47723a1b1'},
 'feeProvider': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/feeProvider.zip',
  'SCORE': 'cxd66f32f94600843c9441f504ba6ee075d4387bc8'},
 'lendingPoolCore': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/lendingPoolCore.zip',
  'SCORE': 'cx5950ae341b07b9d557ed545e45e4f2e3a506ad9a'},
 'oToken': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/oToken.zip',
  'SCORE': 'cx5e71af8b721dfe96af95f03cb514bfa04480180a'},
 'priceOracle': {'zip': '/home/shubham/Work/ICON/openmoneymarket-mono/score/priceOracle.zip',
  'SCORE': 'cx8397f621adac63d786c24cfdf920e683ae320dfd'},
 'sample_token': {'zip': '/home/shubham/Work/ICON/openmoneymarket-

# Updating a contract

In [36]:
contract = 'lendingPool'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contracts[contract]["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xa77c8cd5457c93c267a472e685a67825802efaef9e4902dd77be6ea7b0a29c5b',
 'blockHeight': 96428,
 'blockHash': '0xd805acfe7df5115318a02ea7e339dd6fcb55ff378577fc921aeaebc1d622876b',
 'txIndex': 1,
 'to': 'cxad29953f5b8957704e9092632bccc3b1aac62eea',
 'scoreAddress': 'cxad29953f5b8957704e9092632bccc3b1aac62eea',
 'stepUsed': 1635708000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1635708000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

# Configuring SCOREs's parameters

In [22]:
settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setUSDbAddress', 'params':{'_address': contracts['sample_token']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']['SCORE']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 25*10**14}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token']['SCORE'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']['SCORE']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']['SCORE']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']['SCORE']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']['SCORE']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']['SCORE']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token']['SCORE'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"1{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"5{'0'*17}"} ]}}]

In [23]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'status': 1}


Calling setOracleAddress, with parameters {'_addr

## Adding new reserve in LendingPoolCore



In [25]:
params ={"_reserve": {"reserveAddress":contracts['sample_token']['SCORE'],"oTokenAddress":contracts['oToken']['SCORE'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"60000000000000000000","liquidationThreshold":"65000000000000000000","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'txHash': '0xa3fb756b7da6af80ace251eea1031fd22775ca800232bec427ead404eb77d11a',
 'blockHeight': 488856,
 'blockHash': '0xd73d1654464bfa7fd9f7c6e63966bc37d5da852ca49a7341f366fb7f26f2f09e',
 'txIndex': 1,
 'to': 'cx5950ae341b07b9d557ed545e45e4f2e3a506ad9a',
 'stepUsed': 273040,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 273040,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

# Transfering USDb to a wallet

In [ ]:
params = {"_to": "", "_value": 1000000000000000000000,}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Depositing USDb from a wallet

In [67]:
data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool']['SCORE'], "_value": 1000000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xb1fc5062c7d5fa859a08dd5b505b322bd03450067aec12f06e456c79c25c7d26',
 'blockHeight': 443726,
 'blockHash': '0x1bcfa6746677749da2e16fb7b42ff3708e2dcb3a9fcb7f6a35458f58c72ea4e6',
 'txIndex': 1,
 'to': 'cx2c002610b8aeaff84f6cb97c28294b1986016143',
 'stepUsed': 729440,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 729440,
 'eventLogs': [{'scoreAddress': 'cx879e276135a6f25236a8abf2e4c9801887538f60',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx2c002610b8aeaff84f6cb97c28294b1986016143',
    '0x0',
    '0x0'],
   'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']},
  {'scoreAddress': 'cx2f9c6ae40e6ae4c8d39961b3f3d701363b361773',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x0']},
  {'scoreAddress': 'cx2f9c6ae40e6ae4c8d39961b3f3d701363b361773',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x3635c9adc5dea00000']},
  {'scoreAddress': 'cx2f9c6ae40e6ae4

# Borrowing USDb

In [75]:
params ={"_reserve": contracts['sample_token']['SCORE'], "_amount":150*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx9a1e70f60ee138d8e2c2eee0daf28a7d0645bd02', 'stepLimit': '0x11a350', 'timestamp': '0x5b3ab99c871a5', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cx2c002610b8aeaff84f6cb97c28294b1986016143', '_amount': '0x821ab0d4414980000'}}, 'signature': 'nnBAhBv6BGeWZE63Rr6d02x+c/R36yjf49WvR/G5VLd6hz7tKN6lL6lG/kghrjVg0ZPWIf2KdJi+rmBPoRrFWAA='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x4f24f998ab722bbcdd6589d23f1bd67cc98a4f30ff863710b291f60545017e0f',
 'blockHeight': 443982,
 'blockHash': '0x795f4716b2053eca2b1b8585c02c10fe30705843841760035bf91b80abb581c9',
 'txIndex': 1,
 'to': 'cx9a1e70f60ee138d8e2c2eee0daf28a7d0645bd02',
 'stepUsed': 1055920,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1055920,
 'eventLogs': [{'scoreAddress': 'cx879e276135a6f25236a8abf2e4c9801887538f60',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx2c002610b8aeaff84f6cb97c28294b1986016143',
    '0x864ae65482800',
    '0x3e2c284391c000'],
   'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']},
  {'scoreAddress': 'cx2c002610b8aeaff84f6cb97c28294b1986016143',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx879e276135a6f25236a8abf2e4c9801887538f60',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    '0x821ab0d4414980000'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x0

# Reedeming/Withdrawing USDb

In [ ]:
params ={"_amount": 10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE']) \
    .nid(NID) \
    .nonce(100) \
    .method("redeem")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Repaying USDb

In [ ]:
data = "{\"method\": \"repay\", \"params\": {\"amount\": 5000000000000000000}}".encode("utf-8")
params = {"_to": LendingPool, "_value": 5000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(caller_wallet.get_address())\
    .to(contracts['sample_token']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, caller_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

# Readonly methods to get data

### Getting specific reserve data

In [26]:
#this returns the reserve data of USDb
params = {'_reserve': contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': '0x0',
 'baseLTVasCollateral': 60.0,
 'borrowCumulativeIndex': 1.0,
 'borrowRate': '0x0',
 'borrowingEnabled': '0x1',
 'decimals': 1.8e-17,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': '0x0',
 'liquidationBonus': 1e-17,
 'liquidationThreshold': 65.0,
 'liquidityCumulativeIndex': 1.0,
 'liquidityRate': '0x0',
 'oTokenAddress': 'cx5e71af8b721dfe96af95f03cb514bfa04480180a',
 'reserveAddress': 'cx3975f669579804bb7e05e954a66626095a5079f8',
 'totalBorrows': '0x0',
 'totalLiquidity': '0x0',
 'usageAsCollateralEnabled': '0x1'}


### Getting data of all reserves

In [50]:
#this returns the reserve data of all the reserves
params = {'_reserve': contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if v=="0x1" or v =="0x0" or "Address" in k:
            continue
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'availableLiquidity': '0x0',
          'baseLTVasCollateral': 60.0,
          'borrowCumulativeIndex': 1.0,
          'borrowRate': '0x0',
          'borrowingEnabled': '0x1',
          'decimals': 1.8e-17,
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': '0x0',
          'liquidationBonus': 1e-17,
          'liquidationThreshold': 65.0,
          'liquidityCumulativeIndex': 1.0,
          'liquidityRate': '0x0',
          'oTokenAddress': 'cx5e71af8b721dfe96af95f03cb514bfa04480180a',
          'reserveAddress': 'cx3975f669579804bb7e05e954a66626095a5079f8',
          'totalBorrows': '0x0',
          'totalLiquidity': '0x0',
          'usageAsCollateralEnabled': '0x1'}}


### Getting user account data

In [27]:
#returns the user data of all the reserves combined
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'currentLiquidationThreshold': 0.0,
 'currentLtv': 0.0,
 'healthFactor': -1e-18,
 'healthFactorBelowThreshold': 0.0,
 'totalBorrowBalanceUSD': 0.0,
 'totalCollateralBalanceUSD': 0.0,
 'totalFeesUSD': 0.0,
 'totalLiquidityBalanceUSD': 0.0}


### Getting user reserve data

In [28]:
#returns the data of a specific user for a specific reserve
#this returns the data of user for USDb reserve
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' ,"_reserve":contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.0,
 'currentBorrowBalance': 0.0,
 'currentBorrowBalanceUSD': 0.0,
 'currentOTokenBalance': 0.0,
 'currentOTokenBalanceUSD': 0.0,
 'lastUpdateTimestamp': 0,
 'liquidityRate': 0.0,
 'originationFee': 0.0,
 'principalBorrowBalance': 0.0,
 'principalBorrowBalanceUSD': 0.0,
 'useAsCollateral': 0.0,
 'userBorrowCumulativeIndex': 0.0}


### Getting the data of user for all reserves

In [51]:
#returns the data of a specific user for all reserves

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getUserAllReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'borrowRate': 0.0,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 0.0,
          'currentOTokenBalanceUSD': 0.0,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 0.0,
          'originationFee': 0.0,
          'principalBorrowBalance': 0.0,
          'principalBorrowBalanceUSD': 0.0,
          'useAsCollateral': 0.0,
          'userBorrowCumulativeIndex': 0.0}}


### Get reserve configurations


In [53]:
#returns the configurations of a specific reserve
params = {"_reserve":contracts['sample_token']['SCORE']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getReserveConfigurationData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)


{'baseLTVasCollateral': 60000000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 65000000000000000000,
 'usageAsCollateralEnabled': 1}


### Get all reserves configurations

In [56]:
#returns the configurations of all the   reserves

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getAllReserveConfigurationData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        response[key][k]=int(v,0)
pprint(response)


{'USDb': {'baseLTVasCollateral': 60000000000000000000,
          'borrowingEnabled': 1,
          'decimals': 18,
          'isActive': 1,
          'liquidationBonus': 10,
          'liquidationThreshold': 65000000000000000000,
          'usageAsCollateralEnabled': 1}}


## Realtime USDb balance of user(with accrued interest)

In [33]:

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']['SCORE'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))

0


In [20]:

# params = {'_owner': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider']['SCORE'])\
    .method("getOracleAddress")\
    .build()
response = icon_service.call(_call)
print(response)

cx8397f621adac63d786c24cfdf920e683ae320dfd


In [ ]:
995000000570919367819
995000000564440311889
894000001574533095848
894000000615847612243
894000001574533095848
795000001589272371715
795000001731223349511